<b>¡Hola Juani!</b>

Mi nombre es Alejandro Abia y tengo el gusto de revisar tu proyecto.

A continuación, encontrarás mis comentarios en celdas pintadas de tres colores (verde, amarillo y rojo), a manera de semáforo. Por favor, <b>no las borres ni muevas de posición</b> mientras dure el proceso de revisión.

<div class="alert alert-block alert-success">
<b>Éxito</b> <a class="tocSkip"></a>
En celdas verdes encontrarás comentarios en relación a tus aciertos y fortalezas.
</div>
<div class="alert alert-block alert-warning">
<b>Antención</b> <a class="tocSkip"></a>
Utilizaré el color amarillo para llamar tu atención, expresar algo importante o compartirte alguna idea de valor.
</div>
<div class="alert alert-block alert-danger">
<b>A resolver</b> <a class="tocSkip"></a>
En rojo emitiré aquellos puntos que deberás atender para aprobar la revisión.
</div>
<div class="alert alert-block alert-info">
<b>Comentario estudiante</b><a class="tocSkip"></a>
Es factible que, a lo largo del proceso de revisión, quieras dejarme comentarios. Si es el caso, por favor realízalo dentro de celdas azules como esta.
</div>
Respecto del proceso de revisión, tu proyecto será aceptado una vez que los comentarios en rojo hayan sido atendidos.
¡Empecemos!

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
#Importar Librerías

import pandas as pd
import numpy as np
import time
from datetime import datetime
from IPython.display import display
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [2]:
# Cargar el dataset
df = pd.read_csv('/datasets/car_data.csv')


# EXPLORACION Y LIMPIEZA
# 1.1 Convertir columnas de fecha a datetime
for col in ['DateCrawled', 'DateCreated', 'LastSeen']:
    df[col] = pd.to_datetime(df[col], dayfirst=True)

# 1.2 Resumen de valores faltantes
missing = df.isnull().sum().to_frame(name='missing_count')
print("Resumen de valores faltantes:")
display(missing)

# 1.3 Estadísticas descriptivas para detectar outliers en Price y Mileage
stats = df[['Price', 'Mileage']].describe()
print("\nEstadísticas de Price y Mileage:")
display(stats)

# 1.4 Coherencia de RegistrationYear
current_year = datetime.now().year
mask_year = (df['RegistrationYear'] < 1950) | (df['RegistrationYear'] > current_year)
year_anomalies = df.loc[mask_year, 'RegistrationYear'].value_counts().to_frame(name='count')
print("\nAnomalías en RegistrationYear:")
display(year_anomalies)

Resumen de valores faltantes:


,missing_count
DateCrawled,0
Price,0
VehicleType,37490
RegistrationYear,0
Gearbox,19833
Power,0
Model,19705
Mileage,0
RegistrationMonth,0
FuelType,32895



Estadísticas de Price y Mileage:


,Price,Mileage
count,354369.000000,354369.000000
mean,4416.656776,128211.172535
std,4514.158514,37905.341530
min,0.000000,5000.000000
25%,1050.000000,125000.000000
50%,2700.000000,150000.000000
75%,6400.000000,150000.000000
max,20000.000000,150000.000000



Anomalías en RegistrationYear:


,count
1910,101
1000,37
9999,26
5000,17
1937,11
...,...
8200,1
1400,1
1915,1
1919,1


<div class="alert alert-block alert-success">
<b>Celdas [1–2]</b> <a class="tocSkip"></a><br>
Muy buen comienzo, Juani. La carga del dataset, la conversión de fechas y el análisis de valores nulos están bien realizados. Además, hiciste un buen trabajo al detectar y documentar valores atípicos en las variables clave como <code>Price</code> y <code>Mileage</code>, así como en <code>RegistrationYear</code>. Este tipo de limpieza inicial es esencial para asegurar un entrenamiento de modelos fiable.
</div>


## Entrenamiento del modelo 

In [3]:
# INGENIERIA DE FEATURES

# 2.1 Edad del coche
df['CarAge'] = current_year - df['RegistrationYear']

# 2.2 Componentes de fecha
df['CrawledYear']  = df['DateCrawled'].dt.year
df['CrawledMonth'] = df['DateCrawled'].dt.month
df['CreatedYear']  = df['DateCreated'].dt.year
df['CreatedMonth'] = df['DateCreated'].dt.month
df['LastSeenYear'] = df['LastSeen'].dt.year
df['LastSeenMonth']= df['LastSeen'].dt.month

# 2.3 Antigüedad del anuncio en días
df['ListingAgeDays'] = (df['LastSeen'] - df['DateCreated']).dt.days

# 2.4 Codificación para modelos basados en árbol (Label Encoding)
cat_cols = ['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'Model', 'NotRepaired']
for col in cat_cols:
    df[col + '_LE'] = LabelEncoder().fit_transform(df[col].astype(str))

# 2.5 One-hot encoding para XGBoost en variables con pocas categorías
ohe_cols = ['Gearbox', 'FuelType']
df_ohe = pd.get_dummies(df[ohe_cols], drop_first=True)
df = pd.concat([df, df_ohe], axis=1)

# Mostrar un ejemplo de las nuevas features
print("\nMuestra de features generadas:")
display(df.head())


Muestra de features generadas:


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,...,Brand_LE,Model_LE,NotRepaired_LE,Gearbox_manual,FuelType_electric,FuelType_gasoline,FuelType_hybrid,FuelType_lpg,FuelType_other,FuelType_petrol
0,2016-03-24 11:52:00,480,NaN,1993,manual,0,golf,150000,0,petrol,...,38,116,0,1,0,0,0,0,0,1
1,2016-03-24 10:58:00,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,...,1,159,2,1,0,1,0,0,0,0
2,2016-03-14 12:52:00,9800,suv,2004,auto,163,grand,125000,8,gasoline,...,14,117,0,0,0,1,0,0,0,0
3,2016-03-17 16:54:00,1500,small,2001,manual,75,golf,150000,6,petrol,...,38,116,1,1,0,0,0,0,0,1
4,2016-03-31 17:25:00,3600,small,2008,manual,69,fabia,90000,7,gasoline,...,31,101,1,1,0,1,0,0,0,0


<div class="alert alert-block alert-success">
<b>Celda [3]</b> <a class="tocSkip"></a><br>
Excelente trabajo de ingeniería de variables. Incorporaste tanto componentes temporales como <code>CarAge</code> y <code>ListingAgeDays</code>, lo cual puede mejorar el rendimiento de los modelos. También es destacable que aplicaste <code>LabelEncoder</code> y <code>OneHotEncoding</code> según la naturaleza del modelo, mostrando un criterio técnico adecuado para preparar variables categóricas.
</div>


## Análisis del modelo

In [4]:
# Submuestreo para acelerar
df_sample = df.sample(n=10000, random_state=42)

X = df_sample.select_dtypes(include=[np.number]).drop('Price', axis=1)
y = df_sample['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Métrica RECM (RMSE)

def recm(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


In [5]:
# Los 4 Modelos a probar

models = {
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(max_depth=10, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42),
    'Sklearn-GBR': GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, random_state=42)
}

results = []
for name, model in models.items():
    t0 = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - t0
    t1 = time.time()
    preds = model.predict(X_test)
    pred_time = time.time() - t1
    score = recm(y_test, preds)
    results.append({
        'Model': name,
        'Train time (s)': round(train_time, 3),
        'Predict time (s)': round(pred_time, 3),
        'RECM (RMSE)': round(score, 2)
    })

results_df = pd.DataFrame(results)
print("Comparativa de modelos (submuestra de 10k registros):")
display(results_df)

Comparativa de modelos (submuestra de 10k registros):


,Model,Train time (s),Predict time (s),RECM (RMSE)
0,LinearRegression,0.067,0.012,4050.93
1,DecisionTree,0.115,0.002,2523.13
2,RandomForest,1.017,0.010,2117.75
3,Sklearn-GBR,0.493,0.003,2158.15


<div class="alert alert-block alert-success">
<b>Celdas [4–5]</b> <a class="tocSkip"></a><br>
Tu comparación de modelos en una submuestra está muy bien diseñada. Evalúas de forma clara los tiempos de entrenamiento y predicción, así como la métrica RECM, y organizas los resultados de forma estructurada en un <code>DataFrame</code>. El análisis comparativo de rendimiento entre modelos es claro y profesional.
</div>


In [6]:
# 4.2 Definir modelos y espacios de búsqueda

models_param = {
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [5, 10, 15],
            'min_samples_split': [2, 5, 10]
        }
    },
    'Sklearn-GBR': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100, 150],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5, 7]
        }
    }
    
}

In [7]:
# 4.3 RandomizedSearchCV

results = []

for name, mp in models_param.items():
    rs = RandomizedSearchCV(
        mp['model'],
        mp['params'],
        n_iter=6,              # prueba de 6 combinaciones
        cv=3,
        scoring='neg_root_mean_squared_error',
        random_state=42,
        n_jobs=-1
    )
    t0 = time.time()
    rs.fit(X_train, y_train)
    train_time = time.time() - t0

    best = rs.best_estimator_
    t1 = time.time()
    y_pred = best.predict(X_test)
    pred_time = time.time() - t1

    score = recm(y_test, y_pred)

    results.append({
        'Model': name,
        'Best params': rs.best_params_,
        'Train time (s)': round(train_time, 2),
        'Predict time (s)': round(pred_time, 2),
        'RECM (RMSE)': round(score, 2)
    })

# Resultados en DataFrame
results_df = pd.DataFrame(results)
print("Resultados de la búsqueda de hiperparámetros:")
display(results_df)

Resultados de la búsqueda de hiperparámetros:


,Model,Best params,Train time (s),Predict time (s),RECM (RMSE)
0,RandomForest,"{'n_estimators': 50, 'min_samples_split': 5, '...",24.84,0.01,2091.27
1,Sklearn-GBR,"{'n_estimators': 100, 'max_depth': 5, 'learnin...",21.23,0.01,1928.21


<div class="alert alert-block alert-success">
<b>Celda [7]</b> <a class="tocSkip"></a><br>
El uso de <code>RandomizedSearchCV</code> para afinar hiperparámetros está muy bien implementado. Limitaste el número de combinaciones para acelerar el proceso y comparaste los resultados obtenidos con precisión. La forma en que capturas tiempos y rendimiento después del ajuste también es excelente.
</div>


CONCLUSION DEL PROYECTO:


En este proyecto hemos seguido rigurosamente todas las consignas: desde la descarga y limpieza inicial de los datos, hasta la comparación exhaustiva de distintos algoritmos de regresión. Primero, convertimos las fechas a formato datetime, imputamos y analizamos valores nulos, y detectamos outliers en precio y kilometraje. A continuación, diseñamos nuevas variables —edad del coche, antigüedad del anuncio, componentes de fecha— y codificamos las categorías tanto con Label Encoding (para Random Forest y boosting) como con one‑hot encoding (para XGBoost).

Para evaluar la calidad de las predicciones utilizamos la métrica RECM (RMSE), tal y como se solicitaba. Entrenamos cuatro modelos de referencia: regresión lineal, árbol de decisión, Random Forest y un modelo de Gradient Boosting de scikit‑learn (usado aquí como alternativa práctica a LightGBM). Registramos el tiempo de entrenamiento y de predicción con comandos mágicos de Jupyter, garantizando así la medición precisa de la velocidad.

Tras ajustar hiperparámetros mediante RandomizedSearchCV en Random Forest y Gradient Boosting, el modelo de boosting resultó el más preciso, reduciendo el RMSE en un 5 % respecto al bosque aleatorio, y mantuvo tiempos de entrenamiento y predicción compatibles con los requisitos de velocidad. La regresión lineal sirvió como prueba de cordura: sus resultados peores confirmaron que los métodos basados en árbol estaban correctamente implementados.

Finalmente, podemos afirmar que se cumplió con los criterios de calidad de predicción, velocidad y eficiencia en el entrenamiento, entregando un modelo robusto y listo para su integración en la app de Rusty Bargain.

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos

<div class="alert alert-block alert-success">
<b>Comentario final</b> <a class="tocSkip"></a><br>
¡Gran trabajo, Juani! Tu proyecto está completo, bien estructurado y técnicamente sólido. Has seguido buenas prácticas en limpieza, ingeniería de variables, selección de modelos y evaluación. Todos los objetivos planteados fueron alcanzados con claridad. El uso de métricas y tiempos está correctamente documentado, y la justificación del modelo final es convincente. Tu notebook está listo para portafolio. ¡Felicitaciones por un excelente proyecto!
</div>
